<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/Generalised_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install firebase-admin
import os 
import math
import cv2
import numpy as np 
import csv

In [0]:
#Get Images from Firebase Storage
import datetime
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage

# Fetch the service account key JSON file contents
cred = credentials.Certificate("credentials.json")

# Initialize the app with a service account, granting admin privileges
app = firebase_admin.initialize_app(cred, {
    'storageBucket': 'fir-4ca2c.appspot.com',
}, name='storage')

In [0]:
#Loading E1 - Lenovo data : 504 -- sscb labels 

data = []
reading = []
t = 0
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

imglenovo = []

mits = set()
for blob in blobs:
  if blob.name[7:16]=="lenovoinS" :
    if int(blob.name[-8:-6])<19 and int(blob.name[-8:-6])>5:
      blob.download_to_filename('/content/' + "lenovo" + str(blob.name.split('/')[-1])+ '.jpeg')
      imglenovo.append("lenovo" + str(blob.name.split('/')[-1])+ '.jpeg') 
      print("lenovo" + str(blob.name.split('/')[-1])+ '.jpeg')



In [0]:
print(imglenovo[0])
im_trans=[]
im_ent=[]
im_cont=[]
k=0
for img in imglenovo:
  im = cv2.imread(img)
  im = cv2.resize(im, (32,32))
  im_trans.append(np.ndarray.flatten(transmission(im, retMean = False )))
  im_ent.append(entropy(im))
  im_cont.append(contrast(im))
  k+=1
  print(k)
  
  
print(len(im_trans))
print(len(im_ent))
print(len(im_cont))

In [0]:
dict={}
for i in range(0,505):
  l=[]
  l.append(im_trans[i])
  l.append(im_ent[i])
  l.append(im_cont[i])
  dict[(imglenovo[i])]=l
  

In [0]:
##Loading E2 - Mi data : 760 -- sscb labels 
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

imgminote = []

mits = set()
for blob in blobs:
  if blob.name[7:16]=="minoteSSC" :
    if int(blob.name[-8:-6])<19 and int(blob.name[-8:-6])>5:
        print(blob.name)

        blob.download_to_filename('/content/' + "minote" + str(blob.name.split('/')[-1])+ '.jpeg')
        imgminote.append("minote" + str(blob.name.split('/')[-1])+ '.jpeg') 
        print("minote" + str(blob.name.split('/')[-1])+ '.jpeg')


In [0]:
print(imgminote[0])

for img in imgminote:
  im = cv2.imread(img)
  im = cv2.resize(im, (32,32))
  im_trans.append(np.ndarray.flatten(transmission(im, retMean = False )))
  im_ent.append(entropy(im))
  im_cont.append(contrast(im))
  k+=1
  print(k)
  
  
print(len(im_trans))
print(len(im_ent))
print(len(im_cont))
  

In [0]:
k=0
for i in range(506,1330):
  l=[]
  l.append(im_trans[i])
  l.append(im_ent[i])
  l.append(im_cont[i])
  dict[(imgminote[k])]=l
  k+=1
  


In [0]:
k=0
for key in dict:
  k+=1
  if(k==505):
    print(key)
    print(dict[key])
  if(k==506):
    print(key)
    print(dict[key])
 

lenovo2019-07-02 14:43:53.jpeg
[array([0.46902655, 0.46902655, 0.46902655, ..., 0.98672566, 0.98672566,
       0.98672566]), 33.49928283691406, 55.60050763749207]
minote2019-06-30 18:59:57.jpeg
[array([0.32941176, 0.32941176, 0.31764706, ..., 0.83921569, 0.92941176,
       0.92941176]), 43.589691162109375, 68.69164026918482]


In [0]:
#converting all image timestamps into cpcb compatible timestamp 
tmp=[]
for key in dict:
  print(key[-24:])
  temp=key[-24:-5]
  temp=temp.split('-')
  
  newtemp=temp[2].split(' ')
  
  newesttemp=newtemp[1].split(':')
  
  j=int(newesttemp[1])
  if(j<7.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '00'
  elif(j<15 or j<22.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '15'
  elif(j<30 or j<37.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '30'
  elif(j<45 or j<52.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '45'
  else:
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '00'
  tmp.append(final)
  
  

In [79]:
k=0
cpcb={}
with open('/content/sscb_labels_27_2.csv') as file:
  reader = csv.reader(file, delimiter =',')
  for row in reader:
    k+=1
    if(k>=18):
      cpcb[(row[0])]=row[2]
      
      
print(len(cpcb))


      
    

466


In [78]:
# match={}
# for t in cpcb: 
#   for key in tmp:
#     if(key==t):
#       match[key]=cpcb[t]
      
      
# print(match)
# print(len(match))
      

{'27-06-2019 18:00': '36', '27-06-2019 18:15': '36', '27-06-2019 18:30': '36', '27-06-2019 18:45': '36', '28-06-2019 06:00': '180', '28-06-2019 06:15': '180', '28-06-2019 06:30': '180', '28-06-2019 06:45': '180', '28-06-2019 07:00': '190', '28-06-2019 07:15': '190', '28-06-2019 07:30': '190', '28-06-2019 07:45': '190', '28-06-2019 08:00': '92', '28-06-2019 08:15': '92', '28-06-2019 08:30': '92', '28-06-2019 08:45': '92', '28-06-2019 09:00': '45', '28-06-2019 09:15': '45', '28-06-2019 09:30': '45', '28-06-2019 09:45': '45', '28-06-2019 10:00': '30', '28-06-2019 10:15': '30', '28-06-2019 10:30': '30', '28-06-2019 10:45': '30', '28-06-2019 11:00': '23', '28-06-2019 11:15': '23', '28-06-2019 11:30': '23', '28-06-2019 11:45': '23', '28-06-2019 12:00': '24', '28-06-2019 12:15': '24', '28-06-2019 12:30': '24', '28-06-2019 12:45': '24', '28-06-2019 13:00': '27', '28-06-2019 13:15': '27', '28-06-2019 13:30': '27', '28-06-2019 13:45': '27', '28-06-2019 14:00': '27', '28-06-2019 14:15': '27', '28

ValueError: ignored

In [0]:
def transmission(img, lt = 230, retMean = False):
  ker = np.ones((3, 3))/9.0
  imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  b, g, r = cv2.split(img)
  hsv = cv2.cvtColor(imgrgb, cv2.COLOR_RGB2HSV)
  _, _, v = cv2.split(hsv)
  v_blur = cv2.filter2D(v, -1, ker)
  _,building_mask = cv2.threshold(v_blur, lt, 255, cv2.THRESH_BINARY)
  sky = cv2.bitwise_and(imgrgb, imgrgb, mask = building_mask)
  sky_mask = cv2.bitwise_not(building_mask, building_mask.copy())
  building = cv2.bitwise_and(imgrgb, imgrgb, mask = sky_mask)
 
  al = max(v_blur.flatten())
  #Airlight Found. Now calculate Transmission Map
  img_norm = img/float(al)
  b, g, r = cv2.split(img_norm)
  kernel = np.ones((3,3),np.uint8)
  im_new = np.zeros((img.shape[0], img.shape[1]))
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      im_new[i][j] = min(b[i][j], g[i][j], r[i][j])
  dcp = cv2.erode(im_new, kernel, iterations = 1)
  #ax[3].imshow(im_new, cmap = 'gray')
  #ax[4].imshow(dcp, cmap = 'gray')
  tr = np.ones((dcp.shape[0], dcp.shape[1]))
  for i in range(dcp.shape[0]):
    for j in range(dcp.shape[1]):
      tr[i][j] = 1 - dcp[i][j]
  #ax[1].imshow(tr, cmap = 'gray')
  if retMean == False:
    return tr
  elif retMean == True:
    return np.mean(tr)

In [0]:
def entropy(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = cv2.calcHist( [image.astype('float32')],
              [0], 
              None,
               [256], 
              [0,256] )
  h_norm = 0
  h_norm = cv2.normalize(hist, h_norm)
  ent = 0
  for p in h_norm:
    try:
      ent += p*math.log(p, 2)
    except:
      pass
    
  return (-1 * ent[0])

In [0]:
def contrast(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  s = 0
  ss = 0
  mean = np.mean(image)
  x = (-1 * mean * (np.ones(image.shape)))
  ss = np.square(np.add(image, x))
  cont = np.sqrt(np.sum(ss)/float(image.size))
  return cont

In [0]:
#Reading images from all folders : 
images = []




In [0]:
#Enhancing JPEG timestamp and cpcb timestamp compatibility
mint=[]
for name in imglenovo:
  #print(name)
  mint.append(int(name[-2:]))

print(jpg_names)
print(mint)
new_mint=[]
for j in mint: 
  if(j<7.5):
    new_mint.append('00')
  elif(j<15 or j<22.5):
    new_mint.append('15')
  elif(j<30 or j<37.5):
    new_mint.append('30')
  elif(j<45 or j<52.5):
    new_mint.append('45')
  else:
    new_mint.append('00')

print(len(new_mint))


#compat has original jpeg timestamp corresponding to the new timestamp 
compat={}
k=0
for name in jpg_names:
  compat[name]=str(name[:-2])+str(new_mint[k])
  k+=1 
  
print(compat)

In [0]:
timestamps=[]
for img in imglenovo:
  timestamps.append(img[-24:])
  
print(len(timestamps))
for img in imgminote:
  timestamps.append(img[-24:])
  